In [7]:
import huggingface_hub
huggingface_hub.notebook_login()

# If got error, refer to https://huggingface.co/docs/huggingface_hub/main/en/guides/cli

# Inference 

### Setup

In [ ]:
checkpoint = "cy948/starcoder-perf-airscript"
device = "cuda" # for GPU usage or "cpu" for CPU usage

### Load from Remote source huggingface

Model card on https://huggingface.co/cy948/starcoder-perf-airscript

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, trust_remote_code=True).to(device)

### Load from local



In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

MODEL = 'bigcode/starcoderbase-1b'
PEFT_MODEL = "starcoder-perf-airscript"
# load the original model first
tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    quantization_config=None,
    device_map=None,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
).cuda()

# merge fine-tuned weights with the base model
peft_model_id = f"model/cy948/{PEFT_MODEL}"
model = PeftModel.from_pretrained(base_model, peft_model_id)
model.merge_and_unload()

## Downstream tasks

### Generation

In [ ]:
prompt = """
/* 为图表工作表 Chart1 中分类轴设置标题文本 */
"""

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, trust_remote_code=True).to(device)

inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))

### Fill In Middle

In [ ]:
# User need to completion the code in the middle

prefix = """
/* 为图表工作表 Chart3 中分类轴设置标题文本 */
function test() {     
    let axis = Application.Charts.Item("Chart3").
"""

suffix = """
}
"""

def get_code_completion(prefix, suffix):
    text = prompt = f"""{prefix}{suffix}"""
    model.eval()
    outputs = model.generate(
        input_ids=tokenizer(text, return_tensors="pt").input_ids.cuda(),
        max_new_tokens=128,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.0,
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(get_code_completion(prefix, suffix))